## Paralell FineTune PUCPR/GPT2-Bio-PT with Brateca v1.1

Total of Clinical Notes: 2,855,819

based on: https://huggingface.co/docs/transformers/accelerate

In [4]:
!pip install datasets accelerate transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 KB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 93.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 74.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [6]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, AdamW, get_scheduler
from torch.utils.data import DataLoader
from accelerate import Accelerator
from tqdm import tqdm

In [58]:
dataset = load_dataset('csv', data_files='B1_ClinicalNote.csv', split='train[:10%]+train[-80%:]')
dataset

Dataset({
    features: ['Hospital_ID', 'Patient_ID', 'Admission_ID', 'Note_Date', 'Note_Text', 'Notetaker_Position'],
    num_rows: 899
})

In [32]:
tokenizer = AutoTokenizer.from_pretrained("pucpr/gpt2-bio-pt")

In [59]:
def tokenize_function(examples):
    return tokenizer(examples["Note_Text"], add_special_tokens=True, truncation=True, max_length=256, padding="max_length")

filter_dataset = dataset.filter(lambda example: not example["Note_Text"] == None)
tokenized_datasets = filter_dataset.map(tokenize_function, batched=True, remove_columns=dataset.features.keys())
tokenized_datasets = tokenized_datasets.train_test_split(test_size=0.1)

tokenized_datasets.set_format("torch")

small_train_dataset = tokenized_datasets["train"]
small_eval_dataset = tokenized_datasets["test"]

train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=8)

tokenized_datasets

Filter:   0%|          | 0/899 [00:00<?, ? examples/s]

Map:   0%|          | 0/895 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 805
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 90
    })
})

In [60]:
model = AutoModelForCausalLM.from_pretrained("pucpr/gpt2-bio-pt")

In [61]:
accelerator = Accelerator()
optimizer = AdamW(model.parameters(), lr=3e-5)

train_dataloader, eval_dataloader, model, optimizer = accelerator.prepare(
     train_dataloader, eval_dataloader, model, optimizer
)

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [62]:
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

In [63]:
progress_bar = tqdm(range(num_training_steps))

  0%|          | 0/303 [00:00<?, ?it/s]

In [64]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        input_ids = batch.pop('input_ids').to(device)
        attention_mask = batch.pop('attention_mask').to(device)
        outputs = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        labels=input_ids)
        loss = outputs.loss
        accelerator.backward(loss)
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

100%|██████████| 303/303 [02:53<00:00,  1.98it/s]

In [65]:
prompt = 'O paciente chegou no hospital'
input_tokenized = tokenizer(prompt, return_tensors="pt")
output = model.generate(input_tokenized["input_ids"].to(0))
output_text = tokenizer.decode(output[0].tolist())
print(output_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


O paciente chegou no hospital, sendo tratado com sucesso.  A partir de um estudo de caso,
